<a href="https://colab.research.google.com/github/ikasuryani/Sanbercode0920FinalProject/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd 
trainlink = 'https://raw.githubusercontent.com/ikasuryani/Sanbercode0920FinalProject/main/train.csv'
testlink = 'https://raw.githubusercontent.com/ikasuryani/Sanbercode0920FinalProject/main/test.csv'

df = pd.read_csv(trainlink)

#Encoding
df = pd.get_dummies(df, columns=['Jenis Kelamin', 'Status Perkawinan', 'Kelas Pekerja'])
df = pd.get_dummies(df, columns=['Gaji'], drop_first=True)
df = df.rename(columns={'Gaji_>7jt':'Gaji'})
pddterakhir = {
    '10th': 'SMP',
    '11th': 'SMP',
    '7th-8th': 'SD',
    '9th': 'SD',
    '12th': 'SMP',
    '1st-4th': 'TK',
    '5th-6th': 'TK'
}
df['Pendidikan'] = df['Pendidikan'].replace(pddterakhir)
df['Pekerjaan'].replace("?", float("NaN"), inplace=True)
df.dropna(subset=["Pekerjaan"], inplace=True)
df = pd.get_dummies(df, columns=['Pendidikan', 'Pekerjaan'])

df.head()

,id,Umur,Berat Akhir,Jmlh Tahun Pendidikan,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Jenis Kelamin_Laki2,Jenis Kelamin_Perempuan,Status Perkawinan_Belum Pernah Menikah,Status Perkawinan_Berpisah,Status Perkawinan_Cerai,Status Perkawinan_Janda,Status Perkawinan_Menikah,Status Perkawinan_Menikah LDR,Kelas Pekerja_?,Kelas Pekerja_Pekerja Bebas Bukan Perusahan,Kelas Pekerja_Pekerja Bebas Perusahaan,Kelas Pekerja_Pemerintah Lokal,Kelas Pekerja_Pemerintah Negara,Kelas Pekerja_Pemerintah Provinsi,Kelas Pekerja_Tanpa di Bayar,Kelas Pekerja_Tidak Pernah Bekerja,Kelas Pekerja_Wiraswasta,Gaji,Pendidikan_D3,Pendidikan_D4,Pendidikan_Doktor,Pendidikan_Master,Pendidikan_Pendidikan Tinggi,Pendidikan_SD,Pendidikan_SMA,Pendidikan_SMP,Pendidikan_Sarjana,Pendidikan_Sekolah Professional,Pendidikan_TK,Pekerjaan_?,Pekerjaan_Asisten Rumah Tangga,Pekerjaan_Ekesekutif Managerial,Pekerjaan_Mesin Inspeksi,Pekerjaan_Pembersih,Pekerjaan_Pemuka Agama,Pekerjaan_Penjaga,Pekerjaan_Perbaikan Kerajinan,Pekerjaan_Petani,Pekerjaan_Sales,Pekerjaan_Servis Lainnya,Pekerjaan_Spesialis,Pekerjaan_Supir,Pekerjaan_Tech-support,Pekerjaan_Tentara
0,0,21,242912,9,0.0,0.0,35.0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,49,140782,6,0.0,0.0,40.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2,44,120057,12,61404000.0,0.0,45.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,3,24,194630,13,0.0,0.0,35.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,4,33,219619,14,210336000.0,0.0,40.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [55]:
# df['Kelas Pekerja'].value_counts()
# mask = df['Pendidikan'].isnull()
# print(df[mask])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

X = df.drop('Gaji', axis=1)
y = df['Gaji']
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='roc_auc')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.625354 (0.013719)
LDA: 0.890074 (0.006028)
KNN: 0.672408 (0.017299)
CART: 0.743886 (0.013112)
NB: 0.632696 (0.017832)


In [ ]:
# Make predictions on validation dataset
model = LinearDiscriminantAnalysis()
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)
# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))